In [7]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
from tqdm.notebook import tqdm
import torchvision.transforms as tfs
import pickle


from PIL import Image
from pathlib import Path

from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder



device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu') 


In [8]:
# ===============================
# DATA DOWNLOAD AND UNZIP FROM GOOGLE DRIVE
# ===============================

input_dir = '/kaggle/input/journey-springfield/train/simpsons_dataset/'
rescale_size = 244
batch_size = 32 # 32 - resnet152. 64 - all another

# Imagenet mean and standard (are calculated from all of images)
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

transform = tfs.Compose([
    tfs.Resize((int(rescale_size * 1.25), int(rescale_size * 1.25))),
    tfs.RandomCrop(rescale_size),
    tfs.RandomHorizontalFlip(),
    tfs.ToTensor(),
    tfs.Normalize(imagenet_mean, imagenet_std)
])

dataset_full = torchvision.datasets.ImageFolder(root=input_dir, transform=transform)

# split full dataset
train_idx, valid_idx = train_test_split(list(range(len(dataset_full))), train_size=0.8)
dataset = {
    'train': torch.utils.data.Subset(dataset_full, train_idx),
    'valid': torch.utils.data.Subset(dataset_full, valid_idx)
}

dataset_size = {ds: len(dataset[ds]) for ds in ['train', 'valid']}
dataset_classes = np.array(dataset_full.classes)
print('classes:', dataset_classes, '\nsize', dataset_size)

dataloader = {
    'train': torch.utils.data.DataLoader(
        dataset=dataset['train'], batch_size=batch_size, shuffle=True, num_workers=2
    ),
    'valid': torch.utils.data.DataLoader(
        dataset=dataset['valid'], batch_size=batch_size, shuffle=False, num_workers=2
    ),
}


FileNotFoundError: ignored

In [3]:
!unzip -q "/content/drive/MyDrive/journey-springfield/dataset.zip" -d train_data
!unzip -q "/content/drive/MyDrive/journey-springfield/testset.zip" -d test

unzip:  cannot find or open /content/drive/MyDrive/journey-springfield/dataset.zip, /content/drive/MyDrive/journey-springfield/dataset.zip.zip or /content/drive/MyDrive/journey-springfield/dataset.zip.ZIP.
unzip:  cannot find or open /content/drive/MyDrive/journey-springfield/testset.zip, /content/drive/MyDrive/journey-springfield/testset.zip.zip or /content/drive/MyDrive/journey-springfield/testset.zip.ZIP.
